In [1]:
#Loading the saved_model(change the path according to your directory names)
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
PATH_TO_SAVED_MODEL="/Users/victor/Documents/saved_models"
# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)


#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("data/label_map.pbtxt",use_display_name=True)

#Loading the image
img=['test_images/aug_misc_2.png']

FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /Users/victor/Documents/saved_models/variables/variables
 If trying to load on a different device from the computational device, consider using setting the `experimental_io_device` option on tf.saved_model.LoadOptions to the io_device such as '/job:localhost'.

In [45]:
def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

In [46]:
for image_path in img:
    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)
    input_tensor=tf.convert_to_tensor(image_np)
    input_tensor=input_tensor[tf.newaxis, ...]
    detections=detect_fn(input_tensor)
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
    for key,value in detections.items()}
    detections['num_detections']=num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    image_np_with_detections=image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=3,     
          min_score_thresh=0.1)

Running inference for test_images/aug_misc_2.png... WARNING:tensorflow:8 out of the last 9 calls to <function recreate_function.<locals>.restored_function_body at 0x7fe6ded7a310> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


TypeError: visualize_boxes_and_labels_on_image_array() missing 2 required positional arguments: 'scores' and 'category_index'

In [ ]:
plt.grid(False)
plt.imshow(image_np_with_detections)
plt.show()
#cv2.imwrite("frame.jpg", image)